![Avatar foto de Flávio Coutinho](https://fegemo.github.io/cefet-front-end/images/flavio-avatar.jpg)
# Projeto: Pixel Sides


<dl style="margin-top: 1.5em">
    <dt>Aluno</dt><dd>Flávio Coutinho &lt;<a href="mailto:fegemo@cefetmg.br">fegemo@cefetmg.br</a>&gt;</dd>
    <dt>Turma</dt><dd>Aprendizado de Máquina, UFMG, 01/21</dd>
    <dt>Objetivo</dt><dd>Explorar algoritmos de Aprendizado de Máquina na geração procedural de conteúdo para jogos.</dd>
    <dt>Link</dt><dd><a href="https://fegemo.github.io/pixel-sides/" target="_blank">https://fegemo.github.io/pixel-sides/</a>
    <dt>Vídeo</dt><dd><a href="https://youtu.be/1r4YDKAaCew" target="_blank">no YouTube</a>
</dl>

Conteúdo deste _notebook_:
- [Descrição do Projeto](#Descrição-do-Projeto)
- [Extração de Sprites](#Extração-de-Sprites)
- [Classificador de Pose](#Classificador-de-Pose)
- [Geração de Sprite com DCGAN](#Geração-de-Sprite-com-DCGAN)
- [Geração de um Lado dado Outro](#Geração-de-um-Lado-dado-Outro)

In [6]:
"""
Define uma folha de estilos pro notebook
"""
from IPython.core.display import HTML
def css_styling():
    styles = "<style>" + open("./estilos.css", "r").read() + "</style>"
    return HTML(styles)
css_styling()

## Descrição do Projeto

Este é um projeto para a disciplina de Aprendizado de Máquina do DCC/UFMG em 2021/1, ministrada por Adriano Veloso.
O objetivo é **estudar o uso de algoritmos de aprendizado para a geração de conteúdo procedural para jogos**.
Neste momento, focou-se em imagens de personagens em _pixel art_.

Foi usado o _dataset_ [TinyHero](https://www.kaggle.com/calmness/retro-pixel-characters-generator/), que é composto
por $3648$ arquivos de imagem $(64, 64, 4)$ divididos em $4$ classes, cada uma representando uma direção
para onde o personagem está virado.

![centermaxw600](docs/images/tiny-hero-sides.png)

Os estudos desenvolvidos neste projeto contemplaram uso de redes convolucionais para classificação, geração de novas 
imagens usando redes adversariais convolucionais e a geração da pose de um personagem (direita) dada apenas uma única 
imagem em outra pose (de frente). Também foi feita uma rotina para extrair mais imagens (além do TinyHero) a partir
do software RPG Maker 2000.

O que foi desenvolvido:

0. Coletando imagens do RPG Maker
1. Classificador de pose
2. Gerando novos personagens com uma DCGAN
3. Gerando sprites laterais com Pix2Pix

Este _notebook_ mostra apenas os principais resultados.

## Extração de Sprites

O _notebook_ [00-pick-images-from-rpgmaker.ipynb](00-pick-images-from-rpgmaker.ipynb) contém o código para gerar
um novo _dataset_ de imagens de personagens em _pixel art_ em $4$ direções. Foram geradas $96$ imagens de $(64, 64, 4)$
divididas nas $4$ classes.

![centermaxw300](docs/images/rpgmaker-examples.png)

Esse _minidataset_ foi gerado para avaliar o classificador de poses que foi treinado apenas com os dados do _dataset_ TinyHero pra
verificar o impacto de ter uma distribuição de dados distinta do momento de teste e do de validação. E o resultado foi terrível,
conforme esperado :). Isso será descrito no próximo experimento.

## Classificador de Pose

Veja o _notebook_ [01-pose-classifier-cnn.ipynb](01-pose-classifier-cnn.ipynb) para ver a implementação.
Neste experimento foi desenvolvido um **classificador CNN para estimar a pose** da imagem de um personagem.
A rede gerada possui a seguinte arquitetura:

![center](docs/images/pose-classifier-diagram.svg)

Há 2 sequências de convolução e _max pooling_, seguidas de duas camadas totalmente conectadas. A última tem ativação _softmax_
com 4 saídas e indica a confiança do modelo na classificação de cada exemplo quanto às 4 classes (costas, esquerda, frente, direita).
Ao todo foram menos de $25.000$ parâmetros treináveis.

O treinamento e validação foram feitos no _dataset_ TinyHero em proporção 80/20%:

![centermaxw300](docs/images/tiny-hero-examples.png)

Em apenas 10 épocas a rede foi capaz de atingir **acurácia bem próxima de $100%$** no _dataset_ TinyHero, com o qual foi treinada:

![centermaxw600](docs/images/resultado-classificador-pose.png)



Como um pequeno teste, após ter sido treinada em TinyHero, a rede foi avaliada para classificar o _minidataset_ 
gerado pela coleta de imagens do RPG Maker 2000 (descrito no passo anterior). Ele possui apenas $96$ imagens
e, apesar de ter sido fornecido à rede com um formato idêntico ao TinyHero $(64, 64, 4)$, originalmente suas
imagens tinham $(32, 24, 4)$. Além disso, o estilo artístico é um pouco diferente se comparado ao TinyHero.

Ao fazer a previsão das $96$ imagens quanto a suas poses, os resultados não foram satisfatórios:

![centermaxw700](docs/images/resultado-classificador-rpgmaker.png)

Tanto os exemplos de previsão quanto a matriz de confusão dão ideia dos acertos e erros do classificador nesse _dataset_.

Isso é explicado pelo fato de que a distribuição das imagens no conjunto de treinamento foi radicalmente diferente
da distribuição nesse teste com outro _dataset_.

Ainda assim, é interessante notar que houve vários acertos apesar do classificador nunca ter visto nenhuma das
imagens do RPG Maker.


Em casos assim, é importante:

1. Fazer o _dataset_ de treino conter uma distribuição parecida com a do teste
1. Ter mais dados disponíveis do segundo _dataset_


## Geração de Sprite com DCGAN

O terceiro experimento pode ser visto no _notebook_ [02-generate-new-sprite.ipynb](02-generate-new-sprite.ipynb).
O objetivo foi de treinar uma DCGAN no conjunto de dados TinyHero para gerar novas imagens de personagens
em _pixel art_ nessas $4$ direções.

A rede generativa adversarial convolucional profunda (DCGAN) é dividida em uma rede geradora e outra discriminativa
que competem e evoluem em prol de possibilitar que a geradora consiga produzir imagens semelhantes às do treinamento.

Ambas redes foram relativamente rasas, com ~3 camadas e foram treinadas ao longo de $250$ épocas. Os resultados
não foram muito satisfatórios, conforme pode ser visto no resultado final:

![center](docs/images/dcgan-250.png)

Do ponto de vista da competição entre gerador e discriminador, seus erros ao longo das épocas evoluíram da seguinte forma:

![centermaxw300](docs/images/dcgan-losses.png)

Tanto a arquitetura quanto hiperparâmetros de otimização e inicialização de pesos podem ser experimentados
e acredita-se que existam hipóteses de modelos que fornecerão resultados melhores.


Além disso, com a DCGAN _vanilla_ não é possível definir qual classe de imagem se deseja gerar. Para tanto,
foram propostas as _conditional_ GANs justamente para fornecer um controle de qual classe será gerada.

## Geração de um Lado dado Outro

O quarto e último experimento foi feito no _notebook_ [03-side2side.ipynb](03-side2side.ipynb).
Nele o objetivo foi de usar uma _conditional DCGAN_ para, **dada a imagem de um personagem em uma pose,
gerar a sua imagem em outra das 3 poses restantes**.

Para isso foi usada a arquitetura de redes geradoras Pix2Pix (Isola et al. 2017) com algumas adaptações.
Assim como os demais experimentos, foi usado do _dataset_ TinyHero configurado de forma a
estabelecer pares de imagens de personagens na pose `frente` para `direita`:

![center](docs/images/tiny-hero-front-to-right.png)

A rede geradora foi construída baseada no modelo U-net que reduz a dimensionalidade espacial
da entrada até $(1, 1)$ e depois a reconstrói até o tamanho original, usando _skip connections_
entre as camadas de redução e as de aumento.

Já a rede discriminadora é uma CNN que classifica por _patches_ de $(30, 30)$ da entrada, conforme
previsto pela arquitetura Pix2Pix.

O treinamento foi feito ao longo de $100.000$ passos de processamento de _batch_ (de tamanho $1$) e os
resultados foram bem positivos:

![centermaxw700](docs/images/p2p-result-1.png)

É interessante notar, nesse exemplo, que apenas olhando a imagem de entrada do personagem, não é possível
inferir que ele possui cabelo, nem a presença e cor dos olhos. Mas a rede foi capaz de inferir
esses detalhes possivelmente a partir de exemplos de outros personagens com os quais treinou.

Mais exemplos:

![centermaxw700](docs/images/p2p-result-2.png)
![centermaxw700](docs/images/p2p-result-3.png)

Neste último é interessante notar que a imagem gerada está mais correta do que a esperada (!!). Nitidamente,
no _dataset_ TinyHero as poses `direita` e `esquerda` são um simples espelhamento no eixo Y. Contudo, em
personagens sem simetria por esse eixo, o _dataset_ provê a pose `direita` ou `esquerda` incorreta. E a
rede foi capaz de gerar uma imagem mais correta do que o próprio rótulo.